Simulation of a Quantum Teleportation Circuit

Let's start by defining some gates and STATEs using numpy arrays

In [1]:
import numpy as np

# Define some useful global vars

# STATEs
STATE_0 = np.array([1,0]).reshape((2,1))
STATE_1 = np.array([0,1]).reshape((2,1))
STATE_PLUS = 1/np.sqrt(2) * np.array([1,1]).reshape((2,1))
STATE_MINUS = 1/np.sqrt(2) * np.array([1,-1]).reshape((2,1))
STATE_T = np.array([1/np.sqrt(2), np.exp(1j*np.pi/4)/np.sqrt(2)]).reshape((2,1))

# Numpy function np.kron computes the Kronecker product (a.k.a. tensor product) of two matrices or vectors.
STATE_00 = np.kron(STATE_0,STATE_0)
STATE_01 = np.kron(STATE_0,STATE_1)
STATE_10 = np.kron(STATE_1,STATE_0)
STATE_11 = np.kron(STATE_1,STATE_1)
STATE_PLUS_0 = np.kron(STATE_PLUS, STATE_0)
STATE_PLUS_PLUS = np.kron(STATE_PLUS, STATE_PLUS)

BELL_00 = 1/np.sqrt(2) * np.array([1,0,0,1]).reshape((4,1))
BELL_01 = 1/np.sqrt(2) * np.array([0,1,1,0]).reshape((4,1))
BELL_10 = 1/np.sqrt(2) * np.array([1,0,0,-1]).reshape((4,1))
BELL_11 = 1/np.sqrt(2) * np.array([0,1,-1,0]).reshape((4,1))


# Pauli matrices
X = np.array([[0,1],[1,0]])
Y = np.array([[0, -1j],[1j, 0]])
Z = np.array([[1,0],[0,-1]])

# Identity matrix
I = np.identity(2)

# Controlled gates
#CNOT = np.array([[1,0,0,0],[0,1,0,0],[0,0,0,1],[0,0,1,0]])
#CZ = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,-1]])

# Hadamard gate
H = 1/np.sqrt(2) * np.array([[1,1],[1,-1]])

Now, let's apply the gates to these STATEs to check they return what's expected

In [2]:
# apply the gates to the STATEs
print(f'X|0>={X @ STATE_0}\nY|1>={Y @ STATE_1}\nZ|+>={Z @ STATE_PLUS}\nY|T>={Y @ STATE_T}')

X|0>=[[0]
 [1]]
Y|1>=[[0.-1.j]
 [0.+0.j]]
Z|+>=[[ 0.70710678]
 [-0.70710678]]
Y|T>=[[0.5-0.5j       ]
 [0. +0.70710678j]]


Apply CNOT and CZ gates to |00>, |+0>, |++>

In [ ]:
# apply CNOT and CZ gates to |00>, |+0>, |++>
print(f'STATEs are defined as:\
      \n|00>={STATE_00}\
      \n|11>={STATE_11}\
      \n|+0>={STATE_PLUS_0}\
      \n|++>={STATE_PLUS_PLUS}\n')

In [ ]:
def circuit_1q(in_STATE: np.array, gates: list) -> np.array:
    """
    Apply a list of gates to a single qubit STATE.
    
    Parameters:
    in_STATE (np.array): Input STATE vector.
    gates (list): List of gate matrices to apply.
    
    Returns:
    np.array: Resulting STATE after applying the gates.
    """
    STATE = in_STATE
    for gate in gates:
        STATE = gate @ STATE
    return STATE

Define some auxiliary functions to implement circuits

In [6]:
def kron_concat(op_list):
    """
    Create a Kronecker product of a matrix with itself n times.
    
    Parameters:
    matrix (np.array): The input matrix.
    n (int): The number of times to apply the Kronecker product.
    
    Returns:
    np.array: The resulting matrix after applying the Kronecker product n times.
    """
    result = op_list[0]
    for op in op_list[1:]:
        result = np.kron(result, op)
    return result

def measure(state_in: np.ndarray,
            n: int =0) -> tuple:
    """
    Measure a quantum state and return the outcome and the basis component where it was projected.
    """
    dim = state_in.shape[0]
    n_qbits = int(np.log2(dim))
    projector = {'0': STATE_0 @ STATE_0.T, '1': STATE_1 @ STATE_1.T}
    projector_global = {'0': kron_concat([I]*n + [projector['0']] + [I]*(n_qbits-n-1)),
                        '1': kron_concat([I]*n + [projector['1']] + [I]*(n_qbits-n-1))}

    state_0 = projector_global['0'] @ state_in
    state_1 = projector_global['1'] @ state_in

    out = np.random.choice([0,1], p = [np.linalg.norm(state_0)**2, np.linalg.norm(state_1)**2])

    operator = kron_concat([I]*n + [projector[str(out)]] + [I]*(n_qbits-n-1))

    state_out = operator @ state_in

    # Renormalize
    state_out /= np.linalg.norm(state_out)
    
    return out, state_out


def controlled_op(state_in: np.ndarray, 
                  op: np.ndarray, 
                  control: int = 0, 
                  target: int =1) -> np.ndarray:
    """
    Apply a controlled-op gate (CNOT, CZ, ...) with specified control and target qubits, not
    necessairly consecutive.
    """
    dim = state_in.shape[0]
    n_qbits = int(np.log2(dim))
    projector = projector = {'0': STATE_0 @ STATE_0.T, '1': STATE_1 @ STATE_1.T}

    op0 = kron_concat([I]*control + [projector['0']] + [I]*(n_qbits-control-1))

    if control < target:
        op1 = kron_concat([I]*control + [projector['1']] + [I]*(target-control-1) + [op] + [I]*(n_qbits-target-1))
    else:
        op1 = kron_concat([I]*target + [op] + [I]*(control-target-1) + [projector['1']] + [I]*(n_qbits-control-1))

    state_out = (op0 + op1) @ state_in

    return state_out
    

Now, define the Quantum teleportation Circuit function

In [7]:
def qtelep_circuit(state_in: np.array):
    """
    Quantum teleportation circuit function.
    
    Parameters:
    state_in (np.array): Input quantum state to be teleported.
    
    Returns:
    np.array: The teleported quantum state.
    """

    # Step 0: Prepare the input state CAB
    state_0 = np.kron(state_in, BELL_00)

    # Step 1: Apply CNOT to C->A
    state_1 = controlled_op(state_0, X, control=0, target=1)
    print(f'State 1:\n{state_1}')

    # Step 2: Apply Hadamard to C
    op = kron_concat([H]+[I]*2)
    state_2 = op @ state_1
    print(f'State 2:\n{state_2}')

    # Step 3: Measure qubit A
    outcome, state_3 = measure(state_2, 1)
    print(f'Outcome of measurement on qubit A: {outcome}')
    print(f'State 3 after measurement:\n{state_3}')

    # Step 4: Measure qbit C
    outcome, state_4 = measure(state_3, 0)
    print(f'Outcome of measurement on qubit C: {outcome}')
    print(f'State 4 after measurement:\n{state_4}')

    # Step 5: Apply CNOT to A->B
    state_5 = controlled_op(state_4, X, control=1, target=2)
    print(f'State 5 after CNOT A->B:\n{state_5}')
    print(f'Final state before applying CZ:\n{state_5}')

    # Step 6: Apply CZ to C->B
    state_6 = controlled_op(state_5, Z, control=0, target=2)
    print(f'Final state after applying CZ C->B:\n{state_6}')
    
    return state_6

In [8]:
state_in = STATE_MINUS # Example input state
teleported_state = qtelep_circuit(state_in)
#print(f'Teleported state:\n{teleported_state}')

State 1:
[[ 0.5]
 [ 0. ]
 [ 0. ]
 [ 0.5]
 [ 0. ]
 [-0.5]
 [-0.5]
 [ 0. ]]
State 2:
[[ 0.35355339]
 [-0.35355339]
 [-0.35355339]
 [ 0.35355339]
 [ 0.35355339]
 [ 0.35355339]
 [ 0.35355339]
 [ 0.35355339]]
Outcome of measurement on qubit A: 0
State 3 after measurement:
[[ 0.5]
 [-0.5]
 [ 0. ]
 [ 0. ]
 [ 0.5]
 [ 0.5]
 [ 0. ]
 [ 0. ]]
Outcome of measurement on qubit C: 0
State 4 after measurement:
[[ 0.70710678]
 [-0.70710678]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]]
State 5 after CNOT A->B:
[[ 0.70710678]
 [-0.70710678]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]]
Final state before applying CZ:
[[ 0.70710678]
 [-0.70710678]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]]
Final state after applying CZ C->B:
[[ 0.70710678]
 [-0.70710678]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]]
